In [30]:
!pip install geopy

   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ------------------- ------------------- 61.4/125.4 kB 656.4 kB/s eta 0:00:01
   -------------------------------------- 125.4/125.4 kB 921.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB 1.9 MB/s eta 0:00:00


In [1]:
import requests
import geopy
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np

In [2]:
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'

In [3]:

url = "https://api.yelp.com/v3/businesses/business_id_or_alias/reviews?limit=20&sort_by=yelp_sort"
api_key = "XhwAXVpj3nrNgikNQREOaW4XXyVvMBwQV_9L9mKp-kIawTch1UpoXOBhWPTr9Qnvar-FCz_8u4Dic3DMDXIy1o69Mm-Tj-hEFtTo5x84wY8YxxXjFhlVn4K9CFfjZXYx"
headers = {
    "accept": "application/json",
    'Authorization': 'Bearer %s' % api_key
}

response = requests.get(url, headers=headers)

print(response.text)

{"error": {"code": "BUSINESS_NOT_FOUND", "description": "The requested business could not be found."}}


In [4]:
# DEFAULT_TERM = 'dinner'
# DEFAULT_LOCATION = 'San Francisco, CA'
# SEARCH_LIMIT = 3

In [5]:
def search_yelp(api_key, term, location):
    
    endpoint = "https://api.yelp.com/v3/businesses/search"
    headers = {
        "Authorization": f"Bearer {api_key}"
    }
    for offset in range(0,1000,20):
        params = {
            "term": term.replace(' ', '+'),
            "location": location.replace(' ', '+'),
            "limit": 20,
            "offset": offset
        }
        response = requests.get(url,headers = headers, params = params)

    # Check if the request was successful (status code 200)
        if response.status_code == 200:
        # Parse the JSON response
            data+= response.json()['businesses']
        
    #df = pd.json_normalize(data)
        # Extract and print relevant information for the top 20 restaurants
        # for index, business in enumerate(data.get("businesses", []), start=1):
        #     print(f"{index}. Name: {business.get('name')}")
        #     print(f"   Rating: {business.get('rating')}")
        #     print(f"   Address: {', '.join(business.get('location', {}).get('display_address', []))}")
        #     print(f"   Phone: {business.get('phone')}")
        #     print(f"   ID: {business.get('id')}")
        #     print(f"   The category is: {business.get('categories')}")
        #     print(f"   is_open_now: {business.get('is_open')}")
        #      # print("   The review count is: ",business.get("review_count"))
        #      # print("-----")
    else:
         print(f"Error: {response.status_code}, {response.text}")
        
search_yelp(api_key, 'term', 'IL')


Error: 400, {"error": {"code": "VALIDATION_ERROR", "description": "['20', '20'] is not of type 'integer'", "field": "limit", "instance": ["20", "20"]}}


In [114]:
def get_businesses(location, term, api_key):
    headers = {'Authorization': 'Bearer %s' % api_key}
    url = 'https://api.yelp.com/v3/businesses/search'

    data = []
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': location.replace(' ', '+'),
            'term': term.replace(' ', '+'),
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break

    return data
get_businesses(, term, api_key)

NameError: name 'location' is not defined

In [39]:
# calling the Nominatim tool and create Nominatim class
loc = Nominatim(user_agent="Geopy Library")

# entering the location name
getLoc = loc.geocode("1303 W Elmdale Ave")

# printing address
print(getLoc.address)

# printing latitude and longitude
print("Latitude = ", getLoc.latitude)
print("Longitude = ", getLoc.longitude)

1303, West Elmdale Avenue, Edgewater Glen, Edgewater, Chicago, Lake View Township, Cook County, Illinois, 60660, United States
Latitude =  41.99080446647168
Longitude =  -87.66293008241215


In [56]:
data = response.json()
for i in range(len(data)):
    print(data[i])

KeyError: 0

In [14]:
# empty list to place our data for each page
lst = []
API_KEY = "XhwAXVpj3nrNgikNQREOaW4XXyVvMBwQV_9L9mKp-kIawTch1UpoXOBhWPTr9Qnvar-FCz_8u4Dic3DMDXIy1o69Mm-Tj-hEFtTo5x84wY8YxxXjFhlVn4K9CFfjZXYx"
# our offset parameter - each page 50 rows
offset = 0
print('initial offset number: {}'.format(offset))

# loop through the api 20 times (limit is 1000 rows with each page includes 50 rows)
for i in range(20):
    
    try:
        headers = {'Authorization': 'Bearer {}'.format(API_KEY)}
        search_api_url = 'https://api.yelp.com/v3/businesses/search'
        params = {'term': 'food', 
                  'categories': 'restaurants',
                  'location': 'NYC',
                  'offset': offset,
                  'limit': 50}

        response = requests.get(search_api_url, 
                                headers=headers, 
                                params=params, 
                                timeout=10)

        # return a dictionary
        data_dict = response.json()
        
        # convert the business dictionary to a pandas dataframe and append to list
        data = pd.DataFrame(data_dict['businesses'])
        lst.append(data)
      
        # add 50 to the offset to access a new page
        offset += 50
        print('current offset number: {}'.format(offset))
        
    except Exception as ex:
        print('exception: {}\nexit loop.'.format(ex))
        break

# concatenate all pages to one dataframe and reset index
df = pd.concat(lst)
df = df.reset_index(drop=True)

# review shape of dataframe
rows, columns = df.shape
print()
print('query includes {:,} rows and {} columns.'.format(rows, columns))
print('row id is unique: {}.'.format(df['id'].is_unique))

# review if dataframe id is unique, if not drop duplicates
if df['id'].is_unique == False:
    duplicates = df.loc[df.duplicated(subset=['id'])]
    vals = list(duplicates.head()['name'].values)
    print('\nduplicates found: {}.'.format(vals))
    
    df = df.drop_duplicates(subset=['id']).reset_index(drop=True)
    print('dropping duplicates...')
    
    rows, columns = df.shape
    print('\nrow id is unique: {}.'.format(df['id'].is_unique))
    print('query includes {:,} rows and {} columns.'.format(rows, columns))

initial offset number: 0
current offset number: 50
current offset number: 100
current offset number: 150
current offset number: 200
current offset number: 250
current offset number: 300
current offset number: 350
current offset number: 400
current offset number: 450
current offset number: 500
current offset number: 550
current offset number: 600
current offset number: 650
current offset number: 700
current offset number: 750
current offset number: 800
current offset number: 850
current offset number: 900
current offset number: 950
current offset number: 1000

query includes 1,000 rows and 16 columns.
row id is unique: False.

duplicates found: ['Taiwanese Yummy', 'Peri Peri Grill House', 'Moono', 'Celestine', 'Tonchin'].
dropping duplicates...

row id is unique: True.
query includes 903 rows and 16 columns.


In [29]:

df[["latitude","longitude"]] = df["coordinates"].apply(pd.Series)
#df[["address1","address2"]] = df["location"].apply(pd.Series)
max_num_addresses = df['location'].apply(lambda x: len(x) if isinstance(x, dict) else 0).max()

for i in range(1, max_num_addresses + 1):
    col_name = f'address_{i}'
    df[col_name] = df['location'].apply(lambda x: x[f'address{i}'] if isinstance(x, dict) and f'address{i}' in x else None)

df.head()
none_count = df.isna().sum()
print(none_count)

id                 0
alias              0
name               0
image_url          0
is_closed          0
url                0
review_count       0
categories         0
rating             0
coordinates        0
transactions       0
price            399
location           0
phone              0
display_phone      0
distance           0
longitude          0
latitude           0
address_1          1
address_2        314
address_3        417
address_4        903
address_5        903
address_6        903
address_7        903
address_8        903
dtype: int64
